# Ballbot Wall Pushing Task

In [1]:
import Pkg; 
Pkg.activate(@__DIR__); 
Pkg.instantiate()

  Activating environment at `~/Documents/PhD/courses/16745_optimal_control_rl/project/ballbot_ocrl_navigation/Project.toml`


In [2]:
using RigidBodyDynamics
using LinearAlgebra
using StaticArrays
using MeshCat
using MeshCatMechanisms
using ForwardDiff
using Plots
using RobotDynamics
using Ipopt
using GeometryBasics: HyperRectangle, Vec

import ECOS
import Convex as cvx
import MathOptInterface as MOI
using FileIO, JLD2;

import MeshCat as mc

WebIO._IJuliaInit()

In [3]:
include(joinpath(@__DIR__, "utils","fmincon.jl"))
include(joinpath(@__DIR__, "utils","ballbot_model.jl"))

rk4 (generic function with 1 method)

Obtain a valid wall pushing pose

In [4]:
bb_arms = Ballbot_arms()
contact_state = MechanismState(bb_arms.mech)

q = zeros(9)
q[1] = -0.5 # move to a distance
q[3] = -0.10 # lean forward
q[6] = 0.785 # right shoulder
q[7] = -0.785 # left shoulder
q[8] = 1.5708 # right elbow 90 deg
q[9] = -1.5708 # left elbow 90 deg
RigidBodyDynamics.set_configuration!(contact_state, q)
RigidBodyDynamics.set_velocity!(contact_state, zeros(9))
world = RigidBodyDynamics.findbody(bb_arms.mech, "world")

RigidBody: "world"

In [5]:
right_end = RigidBodyDynamics.findbody(bb_arms.mech, "RArm4")
right_end_tf = RigidBodyDynamics.frame_definitions(right_end)[end].from
r_end_eff_pos = transform(contact_state, Point3D(right_end_tf, zero(SVector{3})), default_frame(world))

left_end = RigidBodyDynamics.findbody(bb_arms.mech, "LArm4")
left_end_tf = RigidBodyDynamics.frame_definitions(left_end)[end].from
l_end_eff_pos = transform(contact_state, Point3D(left_end_tf, zero(SVector{3})), default_frame(world))

@show r_end_eff_pos
@show l_end_eff_pos

r_end_eff_pos = Point3D in "world": [0.27577002126465655, 1.115338121461541, 1.3907916486581435]
l_end_eff_pos = Point3D in "world": [-0.275769990284157, 1.1153381070021942, 1.3907916773182019]


Point3D in "world": [-0.275769990284157, 1.1153381070021942, 1.3907916773182019]

Visualize what a wall pushing pose looks like

In [6]:
state = MechanismState(bb_arms.mech)
RigidBodyDynamics.set_configuration!(state, q)
RigidBodyDynamics.set_velocity!(state, zeros(9))
ts, qs, vs = simulate(state, 0.1, Δt = 1e-2);

In [7]:
vis = Visualizer()
display(render(vis))
mvis = MechanismVisualizer(bb_arms.mech, URDFVisuals(urdf_arms), vis["test"])
box = HyperRectangle(Vec(-2.5, r_end_eff_pos.v[2], 0), Vec(5., 0.1, 5))
setobject!(vis["wall"], box)

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
└ http://127.0.0.1:8703


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), Set{Any}(), ip"127.0.0.1", 8703))

MeshCat Visualizer with path /meshcat/wall at http://127.0.0.1:8703

In [8]:
MeshCatMechanisms.animate(mvis, ts, qs; realtimerate = 0.1);

## Trajectory Optimization for Wall Pushing

We use DIRCOL to find a feasible wall-pushing trajectory.

### Dynamics

There are two kind of dynamics modes: contact and non contact.

We schedule the following contact sequence:

$$
\begin{align} 
\mathcal{M}_1 &= \{1\text{:}20,31\text{:}50\} \\
\mathcal{M}_2 &= \{21\text{:}30\}
\end{align}
$$

where $\mathcal{M}_1$ contains the time steps when the BallBot is in free space, and $\mathcal{M}_2$ contains the time steps when the BallBot is in contact with the wall. The jump map sets $\mathcal{J}_1$ and $\mathcal{J}_2$ are the indices where the mode of the next time step is different than the current, i.e. $\mathcal{J}_i \equiv \{k+1 \notin \mathcal{M}_i \; | \; k \in \mathcal{M}_i\}$. We can write these out explicitly as the following:

$$
\begin{align} 
\mathcal{J}_1 &= \{20\} \\
\mathcal{J}_2 &= \{30\}
\end{align}
$$

Both dynamics categories share the same dynamics function `ballbot_dynamics`. However, different ballbot models should be passed to it (generated from `Ballbot_arms()` for free space or `Ballbot_wall()` for when against the wall).

In [9]:
function ballbot_dynamics(model::Ballbot, x::AbstractVector{T1}, u::AbstractVector{T2}) where {T1,T2} 
    T = promote_type(T1,T2)
    state = model.statecache[T]
    res = model.dyncache[T]
    ball_radius = 0.106
    # Convert from state ordering to the ordering of the mechanism
    copyto!(state, x)
    τ = [u[1:2] ./ ball_radius; 0.0; 0.0; u[3:end]]
        
    dynamics!(res, state, τ)
    q̇ = res.q̇
    v̇ = res.v̇
    return [q̇; v̇]
end

function jump1_map(x)
    # elbows experiences inelastic collision
    xn = [x[1:16]; 0.0; 0.0]
    return xn
end

jump1_map (generic function with 1 method)

In [10]:
function create_idx(nx,nu,N) 
    # Our Z vector is [x0, u0, x1, u1, …, xN]
    nz = (N-1) * nu + N * nx # length of Z 
    x = [(i - 1) * (nx + nu) .+ (1 : nx) for i = 1:N]
    u = [(i - 1) * (nx + nu) .+ ((nx + 1):(nx + nu)) for i = 1:(N - 1)]
    
    # Constraint indexing for the (N-1) dynamics constraints when stacked up
    c = [(i - 1) * (nx) .+ (1 : nx) for i = 1:(N - 1)]
    nc = (N - 1) * nx # (N-1)*nx 
    return (nx=nx,nu=nu,N=N,nz=nz,nc=nc,x=x,u=u,c=c)
end

function hermite_simpson(params::NamedTuple, model1::Ballbot, model2::Ballbot, x1::Vector, x2::Vector, u, dt::Real)::Vector
    # Input hermite simpson implicit integrator residual
    f1 = ballbot_dynamics(model1, x1, u)
    f2 = ballbot_dynamics(model2, x2, u)
    xm = 0.5*(x1 + x2) + (dt/8.0)*(f1 - f2)
    fm = ballbot_dynamics(model1, xm, u)
    r = x1 + (dt/6.0)*(f1 + 4*fm + f2) - x2
    return r
end

function rk4(params::NamedTuple, model::Ballbot, x::Vector, u, dt::Float64)
    # Vanilla RK4
    k1 = dt*ballbot_dynamics(model, x, u)
    k2 = dt*ballbot_dynamics(model, x + k1/2, u)
    k3 = dt*ballbot_dynamics(model, x + k2/2, u)
    k4 = dt*ballbot_dynamics(model, x + k3, u)
    x + (1/6)*(k1 + 2*k2 + 2*k3 + k4)
end

function stage_cost(params,x,u,k)
    xref, uref = params.Xref, params.Uref
    Q = params.Q
    R = params.R
    return 0.5*((x-xref[k])'*Q*(x-xref[k])) + 0.5*(u-uref[k])'*R*(u-uref[k])
end

function terminal_cost(params,x)
    Qn = params.Qf
    xref = params.Xref
    return 0.5*((x-xref[end])'*Qn*(x-xref[end]))
end

function ballbot_cost(params::NamedTuple, Z::Vector)::Real
    idx, N = params.idx, params.N 
    # Stage cost
    J = 0 
    for i = 1:(N-1)
        xi = Z[idx.x[i]]
        ui = Z[idx.u[i]]
       
        J += stage_cost(params, xi, ui, i)
    end
    # Terminal cost 
    xend = Z[idx.x[end]]
    J += terminal_cost(params, xend)
    return J 
end

function ballbot_dynamics_constraints(params::NamedTuple, Z::Vector)::Vector
    M1, M2 = params.M1, params.M2
    J1, J2 = params.J1, params.J2
    idx, N, dt = params.idx, params.N, params.dt
    model1, model2 = params.model1, params.model2

    c = zeros(eltype(Z), idx.nc)
    for i = 1:(N-1)
        xi = Z[idx.x[i]]
        ui = Z[idx.u[i]] 
        xip1 = Z[idx.x[i+1]]
        # Apply hermite simpson
        if (i in J1)
            c[idx.c[i]] .= xip1 - jump1_map(rk4(params, model1, xi, ui, dt))
        elseif (i in J2)
            c[idx.c[i]] .= xip1 - rk4(params, model2, xi, ui, dt)
        elseif (i in M1) && !(i in J1)
            c[idx.c[i]] .= xip1 - rk4(params, model1, xi, ui, dt)
        elseif (i in M2) && !(i in J2)
            c[idx.c[i]] .= xip1 - rk4(params, model2, xi, ui, dt)
        end
    end
    return c
end

function ballbot_equality_constraint(params::NamedTuple, Z::Vector)::Vector
    M1, M2 = params.M1, params.M2
    J1, J2 = params.J1, params.J2
    idx, N = params.idx, params.N 
    xg = params.xg
    xic = params.xic
    xcgs = params.xcgs # contact goal constraint
    # Return all of the equality constraint
    c_dynamics = ballbot_dynamics_constraints(params, Z)
    c_contact = zeros(eltype(Z), length(xcgs) * 9)
    for i=1:length(xcgs)
        i_start = (i-1) * 9 + 1
        i_end = i * 9
        c_contact[i_start:i_end] = Z[idx.x[J1[i]][1:9]] - xcgs[i]
    end
    c = [c_dynamics; Z[idx.x[1]] - xic; Z[idx.x[N]] - xg; c_contact]
    return c
end

function ballbot_inequality_constraint(params, Z)
    return zeros(eltype(Z), 0)
end

ballbot_inequality_constraint (generic function with 1 method)

In [11]:
# Optimization Parameters
h = 0.2 # 5 Hz
nq = 9
nx = nq*2     # number of state
nu = 7     # number of controls
Tfinal = 4.0 # final time
Nt = Int(Tfinal/h)   # number of time steps
thist = Array(range(0,h*(Nt-1), step=h));

# Cost terms
diagq = ones(nx)
diagq[1:2] = diagq[1:2]*1 # position cost
diagq[3:4] = diagq[3:4]*1 # lean angles cost

# Cost matrices
Q = diagm(diagq)
Qf = 10*Q
R = 0.5*diagm(ones(nu))

xic = zeros(nx)
xg = zeros(nx)

# Indexing 
idx = create_idx(nx,nu,Nt);

diff_type = :auto

# Create warm start trajectory
Xref = [zeros(nx) for i=1:Nt]
Xfwd = collect(range(xic, [q; zeros(9)], length = 10))
Xwall = [q; zeros(9)]
Xbwd = collect(range([q; zeros(9)], xg, length = 10))
for i=1:10
    Xref[i] .= Xfwd[i]
end

for i=11:20
    Xref[i] .= Xbwd[i-10]
end
Uref = [zeros(7) for i = 1:(Nt-1)];

# Contact scheduling
M1 = [1:10; 11:20]
M2 = []
J1 = [10]
J2 = []

# Models
model1 = Ballbot_arms()
model2 = Ballbot_wall(vec(l_end_eff_pos.v), vec(r_end_eff_pos.v))

# Create params
params = (xic = xic,
          xg = xg,
          xcgs = [q],
          Xref = Xref,
          Uref = Uref,
          dt = h,
          N = Nt,
          idx = idx,
          Q = Q,
          R = R,
          Qf = Qf,
          M1 = M1, 
          M2 = M2,
          J1 = J1, 
          J2 = J2,
          model1 = model1,
          model2 = model2,
          nx = nx,
          nu = nu
          );

# Primal bounds 
x_l = ones(idx.nz)
x_u = ones(idx.nz)

state_l = [
    -2.0, -0.1, # limit the ballbot's movement
    -0.349065850399, -0.349065850399, -3.15, # from manufacturer
    -2.96705972839, -2.96705972839, # from manufacturer
    -0.349065850399, -2.79252680319, # from manufacturer

    -5.0, -5.0,
    -1.0, -1.0, -1.0,
    -3.0, -3.0,
    -3.0, -3.0,
    ]
state_u = [
    2.0, 0.1,
    0.349065850399, 0.349065850399, 3.15,
    2.96705972839, 2.96705972839,
    2.79252680319, 0.349065850399,

    5.0, 5.0,
    1.0, 1.0, 1.0,
    3.0, 3.0,
    3.0, 3.0,
    ]

control_l = -30 * ones(7)
control_u = 30 * ones(7)

for i = 1:(Nt-1)
    x_l[idx.x[i]] .= state_l
    x_u[idx.x[i]] .= state_u
    x_l[idx.u[i]] .= control_l
    x_u[idx.u[i]] .= control_u
end

x_l[idx.x[end]] .= state_l
x_u[idx.x[end]] .= state_u

# Inequality constraint bounds (this is what we do when we have no inequality constraints)
c_l = zeros(0)
c_u = zeros(0)

verbose=true

z0 = zeros(idx.nz)
for i in 1:(Nt-1)
    z0[idx.x[i]] .= Xref[i]
    z0[idx.u[i]] .= Uref[i]
end
z0[idx.x[end]] .= Xref[end];

Z = fmincon(ballbot_cost,ballbot_equality_constraint,ballbot_inequality_constraint,
                x_l,x_u,c_l,c_u,z0,params, diff_type;
                tol = 1e-6, c_tol = 1e-6, max_iters = 100, verbose = verbose);


---------checking dimensions of everything----------
---------all dimensions good------------------------
---------diff type set to :auto (ForwardDiff.jl)----
---------testing objective gradient-----------------
---------testing constraint Jacobian----------------
---------successfully compiled both derivatives-----
---------IPOPT beginning solve----------------------

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:   190791
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian...

In [12]:
X = [Z[idx.x[i]] for i = 1:Nt]
U = [Z[idx.u[i]] for i = 1:(Nt-1)];

X1 = [SVector{9}(x[1:9]) for x in X];
vis = Visualizer()
display(render(vis))
mvis = MechanismVisualizer(bb_arms.mech, URDFVisuals(urdf_arms), vis["ballbot"])
animation = mc.Animation(mvis, thist, X1)
setanimation!(mvis, animation)

box = HyperRectangle(Vec(-2.5, r_end_eff_pos.v[2], 0), Vec(5., 0.1, 5))
setobject!(vis["wall"], box)

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
└ http://127.0.0.1:8704


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), Set{Any}(), ip"127.0.0.1", 8704))

MeshCat Visualizer with path /meshcat/wall at http://127.0.0.1:8704